In [13]:
# Import the required packages
import requests
import duckdb
import pandas as pd
from datetime import datetime, timedelta

In [14]:
# Store the base API information, for this, I'm using NASA's Near Earth Object (NEO) API
api_key_path = "/Users/chriskornaros/Documents/local-scripts/.api_keys/nasa/key.txt"
with open(api_key_path, "r") as file:
    api_key = file.read().strip()

start_date = datetime.strptime("1900-01-01", "%Y-%m-%d")
end_date = start_date + timedelta(days=35)  # Pull data for the first 5 weeks

In [15]:
# Initialize an empty list to store the data
all_data = []

# Iterate through the date range in 7-day increments
current_date = start_date
while current_date < end_date:
    next_date = current_date + timedelta(days=7)
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={current_date.strftime('%Y-%m-%d')}&end_date={next_date.strftime('%Y-%m-%d')}&api_key={api_key}"

    # Fetch data from the API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for date in data["near_earth_objects"]:
            flat_data = pd.json_normalize(data["near_earth_objects"][date])
            all_data.append(flat_data)
    else:
        print(f"Error: {response.status_code}")

    current_date = next_date

# Concatenate all the data into a single DataFrame
if all_data:
    final_data = pd.concat(all_data, ignore_index=True)
else:
    final_data = pd.DataFrame()

In [16]:
#  Open a connection to the persistent database
con = duckdb.connect("test.duckdb")

# Store the data in DuckDB
con.sql("""
    CREATE TABLE IF NOT EXISTS asteroids AS
    SELECT * EXCLUDE(nasa_jpl_url, close_approach_data, 'links.self')
    , unnest(close_approach_data, recursive := true)
    FROM final_data
""")

# Verify the data was stored correctly
result = con.sql("SELECT * FROM asteroids").fetchdf()
print(result)

# Close the connection
con.close()

           id neo_reference_id               name  absolute_magnitude_h  \
0     2068347          2068347  68347 (2001 KB67)                 19.92   
1     2086450          2086450  86450 (2000 CK33)                 18.45   
2     3102762          3102762        (2002 AA29)                 24.10   
3    54244180         54244180         (2022 BU6)                 25.95   
4    54278313         54278313         (2022 JH1)                 22.26   
..        ...              ...                ...                   ...   
238   3648642          3648642        (2013 SM20)                 25.10   
239   3825155          3825155         (2018 LO3)                 25.51   
240   3836662          3836662         (2018 WN2)                 21.20   
241  54017559         54017559          (2020 MX)                 23.95   
242  54421074         54421074          (2024 BH)                 23.22   

     is_potentially_hazardous_asteroid  is_sentry_object  \
0                                 True 

In [313]:
# Flatten the output and then read it with DuckDB to get the column data types
flat_data = pd.json_normalize(data["near_earth_objects"]["2024-12-02"])

raw = con.sql("""CREATE TABLE asteroids AS
                SELECT * EXCLUDE(nasa_jpl_url, close_approach_data, 'links.self')
                , unnest(close_approach_data, recursive := true)
                FROM flat_data
                """)

In [ ]:
flat_data = pd.json_normalize(data["near_earth_objects"]["2024-12-02"])  # Test this
unnested_data = (
    duckdb.sql("""SELECT * EXCLUDE(nasa_jpl_url, close_approach_data, 'links.self')
                                , unnest(close_approach_data, recursive := true)
                                FROM {a}
                                """)
    .fetchdf()
    .format(a=flat_data)
)

In [17]:
con.close()